<a href="https://colab.research.google.com/github/TiwariBro/AutoLLM/blob/main/examples/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/safevideo/autollm/blob/main/examples/quickstart.ipynb)

## 0. Preparation

- Install latest version of autollm and some required packages for this tutorial:

In [1]:
!pip install autollm gradio gitpython nbconvert -Uqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.6/46.6 kB 754.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.4/256.4 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

- Import required modules:

In [ ]:
# import required functions, classes
from autollm import AutoQueryEngine
from autollm.utils.document_reading import read_github_repo_as_documents, read_files_as_documents
import os
import gradio as gr

- Set your OpenAI API key in order to use default gpt-3.5-turbo model:

In [ ]:
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

## 1. Read Files as Documents

- Reading from a Github repo:

In [ ]:
git_repo_url = "https://github.com/ultralytics/ultralytics.git"
relative_folder_path = "docs"   # relative path from the repo root to the folder containing documents
required_exts = [".md"]    # optional, only read files with these extensions

documents = read_github_repo_as_documents(git_repo_url=git_repo_url, relative_folder_path=relative_folder_path, required_exts=required_exts)

Temporary directory created at autollm/temp


INFO:autollm.utils.document_reading:Found 221 'documents'.
INFO:autollm.utils.document_reading:Operations complete, deleting temporary directory autollm/temp..


- Reading from a local folder:

In [ ]:
# required_exts = ...
# documents = read_files_as_documents(input_dir="your/path/to/documents", required_exts=required_exts)

**Note**: If you want to read specific file types, adjust the `required_exts` parameter. By default, our functions will read all [supported file types](https://github.com/run-llama/llama_index/blob/main/llama_index/readers/file/base.py#L20-L37) from the specified source.

## 2. Configuration of AutoQueryEngine

### Basic Usage

- You can completely skip configuration(advanced usage) if you want to use default settings.

- 🌟 **pro tip**: autollm defaults to lancedb as the vector store since it is lightweight, scales from development to production and is 100x cheaper than alternatives!

In [ ]:
query_engine = AutoQueryEngine.from_defaults(documents=documents)

Generating embeddings: 100%|██████████| 520/520 [00:42<00:00, 12.11it/s]


### Advanced Usage

- You can configure the AutoQueryEngine to your needs:

In [ ]:
# llm params
llm_model = "gpt-3.5-turbo"
llm_max_tokens = 512
llm_temperature = 0.1

# service_context_params
system_prompt = """
You are an friendly ai assistant that help users find the most relevant and accurate answers
to their questions based on the documents you have access to.
When answering the questions, mostly rely on the info in documents.
"""
query_wrapper_prompt = '''
The document information is below.
---------------------
{context_str}
---------------------
Using the document information and mostly relying on it,
answer the query.
Query: {query_str}
Answer:
'''
enable_cost_calculator = True
embed_model = "default"  # ["default", "local"]
chunk_size = 512
context_window = 4096

# vector store params
vector_store_type = "LanceDBVectorStore"
lancedb_uri = "./.lancedb"
lancedb_table_name = "vectors"
enable_metadata_extraction = False

# query engine params
similarity_top_k = 3

query_engine = AutoQueryEngine.from_defaults(
    documents=documents,
    llm_model=llm_model,
    llm_max_tokens=llm_max_tokens,
    llm_temperature=llm_temperature,
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    enable_cost_calculator=enable_cost_calculator,
    embed_model=embed_model,
    chunk_size=chunk_size,
    context_window=context_window,
    vector_store_type=vector_store_type,
    lancedb_uri=lancedb_uri,
    lancedb_table_name=lancedb_table_name,
    enable_metadata_extraction=enable_metadata_extraction,
    similarity_top_k=similarity_top_k
)

## 3. Ask Anything to Your Documents

In [ ]:
response = query_engine.query("How to perform tiled inference with SAHI and YOLOv8, include code snippets")

LLM Prompt Token Usage: 1415
LLM Completion Token Usage: 239
LLM Total Token Cost: $0.002601


In [ ]:
print(response.response)

To perform tiled inference with SAHI and YOLOv8, you can use the `get_sliced_prediction` function provided by SAHI. Here is an example code snippet:

```python
result = get_sliced_prediction(
    "path/to/image.jpg",
    detection_model,
    slice_height=256,
    slice_width=256,
    overlap_height_ratio=0.2,
    overlap_width_ratio=0.2
)
```

In this code snippet, you need to replace `"path/to/image.jpg"` with the actual path to your image file. The `detection_model` parameter should be the YOLOv8 model you want to use for object detection.

The `slice_height` and `slice_width` parameters determine the dimensions of each slice. The `overlap_height_ratio` and `overlap_width_ratio` parameters specify the overlap ratios between adjacent slices.

By running this code, SAHI will perform tiled inference on the image using YOLOv8, and the `result` object will contain the predicted bounding boxes and other information.

Please note that you may need to install the necessary dependencies and i

- Or play with it in the gradio app 🚀

In [ ]:
def greet(query):
    return query_engine.query(query).response

demo = gr.Interface(fn=greet, inputs="text", outputs="text")

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


### If you found this project useful, [give it a ⭐️ on GitHub](https://github.com/safevideo/autollm) to show your support!